In [2]:
import torch
import numpy as np
import torch.nn as nn
import os
import time
from torch.utils.data import Dataset, DataLoader
from human_body_prior.body_model.body_model import BodyModel
from human_body_prior.tools.omni_tools import copy2cpu as c2c
from human_body_prior.tools.rotation_tools import aa2matrot,matrot2aa,local2global_pose
import random
from utils import utils_transform
from models.network import AvatarPoser as net
from scipy import signal
from scipy.spatial.transform import Rotation as R
import glob
from IPython import embed
import time
import copy
import pickle
from human_body_prior.body_model.body_model import BodyModel

c:\Users\rnwns\anaconda3\envs\pytorch_geometric\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OUT DATASET JOINTS.   
0 - hip   
1 - abdomen   
2 - chest     
3 - neck      
4 - head     
5 - lefteye    
6 - righteye      
7 - rCollar    
8 - rShldr    
9 - rForeArm      
10 - rHand    
11 - lCollar      
12 - lShldr   
13 - lForeArm    
14 - lHand   
15 - rButtock   
16 - rThigh   
17 - rShin   
18 - rFoot   
19 - lButtock   
20 - lThigh   
21 - lShin   
22 - lFoot    
구조가 AMASS와 다름. 확인 필요.   
   
OUR DATASET PARMS   
name
local
world
infervel
jointsvel
inferlocal
inferworld   

Local : joints local transformation matrix   
World: joints world transformation matrix   
Infervel: pelvis velocity   
Jointsvel: joints velocity    
Inferlocal: pelvis local transformation matrix   
Inferworld: pelvis world transformation matrix   

필요한것 : rotation_local_full_gt_list, hmd_position_global_full_gt_list, body_parms_list, framerate, gender, filepath, position_global_full_gt_world

DATA LOAD AND CHECK

In [2]:
frame_path = './data_fps60/BioMotionLab_NTroje/test/*.pkl'

file_list = glob.glob(frame_path)
print(file_list)
frame_list = []

for paths in file_list:
    for_check = np.load(paths, allow_pickle=True)

    frame = for_check['rotation_local_full_gt_list']
    frame_list.append(frame.shape[0])

print(np.sum(frame_list))

# CMU : 188350
# HDM05 : 46873
# NTroje : 172683


['./data_fps60/BioMotionLab_NTroje/test\\1.pkl', './data_fps60/BioMotionLab_NTroje/test\\10.pkl', './data_fps60/BioMotionLab_NTroje/test\\100.pkl', './data_fps60/BioMotionLab_NTroje/test\\101.pkl', './data_fps60/BioMotionLab_NTroje/test\\102.pkl', './data_fps60/BioMotionLab_NTroje/test\\103.pkl', './data_fps60/BioMotionLab_NTroje/test\\104.pkl', './data_fps60/BioMotionLab_NTroje/test\\105.pkl', './data_fps60/BioMotionLab_NTroje/test\\106.pkl', './data_fps60/BioMotionLab_NTroje/test\\107.pkl', './data_fps60/BioMotionLab_NTroje/test\\108.pkl', './data_fps60/BioMotionLab_NTroje/test\\109.pkl', './data_fps60/BioMotionLab_NTroje/test\\11.pkl', './data_fps60/BioMotionLab_NTroje/test\\110.pkl', './data_fps60/BioMotionLab_NTroje/test\\111.pkl', './data_fps60/BioMotionLab_NTroje/test\\112.pkl', './data_fps60/BioMotionLab_NTroje/test\\113.pkl', './data_fps60/BioMotionLab_NTroje/test\\114.pkl', './data_fps60/BioMotionLab_NTroje/test\\115.pkl', './data_fps60/BioMotionLab_NTroje/test\\116.pkl', './

In [4]:
amass_path = './data_fps60/CMU/test/1.pkl'
npz_path = './support_data/body_models/CMU/01/01_01_poses.npz'

amass_check = np.load(amass_path, allow_pickle=True)
npz_check = np.load(npz_path ,allow_pickle=True)

print(list(npz_check.keys()))
print(list(amass_check.keys()))
print("##############")
#print(npz_check['poses'].shape) # (frame, 156)  // 52(joints) x 3(rotation x y z) 24 body joints, 28 hand joints.
#print(npz_check['trans'].shape) # (frame, 3) 

# 일단 필요한 것.
#print(amass_check['rotation_local_full_gt_list'].shape) # 22 X 6 = 132  // 6DOF 
#print(amass_check['hmd_position_global_full_gt_list'].shape) # 18 x 3 = 54 // position 3, velocity 3, 6, 6
#print(amass_check['body_parms_list'].keys()) # keys : root_orient, pose_body, trans
print(amass_check['body_parms_list']['trans'].shape)
#print(amass_check['head_global_trans_list'][0]) # 4 x 4 matrix
#print(amass_check['position_global_full_gt_world'].shape) # 22 x 3 matrix

print(amass_check['filepath'])

['trans', 'gender', 'mocap_framerate', 'betas', 'dmpls', 'poses']
['rotation_local_full_gt_list', 'hmd_position_global_full_gt_list', 'body_parms_list', 'head_global_trans_list', 'framerate', 'gender', 'filepath', 'position_global_full_gt_world']
##############
torch.Size([2247, 3])
CMU/142/142_19_poses.npz


In [202]:
path = './data_fps60/cmu_full_test.npz'
pickle_data = np.load(path, allow_pickle=True)

np_data_local_file = np.array(pickle_data['local'])
np_data_world_file = np.array(pickle_data['world'])

np_data_infervel = np.array(pickle_data['infervel'])

# 현재 문제
# joint가 다르다. amass:22 , cmu_origninal : 23

In [203]:
support_dir = 'support_data/'
bm_fname_male = os.path.join(support_dir, 'body_models/smplh/{}/model.npz'.format('male'))
dmpl_fname_male = os.path.join(support_dir, 'body_models/dmpls/{}/model.npz'.format('male'))

bm_fname_female = os.path.join(support_dir, 'body_models/smplh/{}/model.npz'.format('female'))
dmpl_fname_female = os.path.join(support_dir, 'body_models/dmpls/{}/model.npz'.format('female'))

num_betas = 16 # number of body parameters
num_dmpls = 8 # number of DMPL parameters
bm_male = BodyModel(bm_fname=bm_fname_male, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=dmpl_fname_male)#.to(comp_device)
bm_female = BodyModel(bm_fname=bm_fname_female, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=dmpl_fname_female)

bm = bm_male

In [206]:
for i in range(len(np_data_local_file)):
    data = dict()

    local = torch.Tensor(np_data_local_file[i]) # 파일별로 분류하기 위해.
    world = torch.Tensor(np_data_world_file[i])

    local_rotmat = local[:, :, :3, :3] # (frame, joint, 3, 3)
    local_rotmat_reshape = local_rotmat.reshape(-1, 3, 3) 
    local_position = local[:, :, :3, 3] # (frame, joint, 3,)
    world_rotmat = world[:, :, :3, :3]
    world_position = world[:, :, :3, 3]
 
    # joint rotation matrix to rotation vector
    local_aa = matrot2aa(local_rotmat_reshape) # (-1, 3)
    local_aa_fix = local_aa.reshape(local_rotmat.shape[0], -1, 3) # (frame, joints, 3)
    local_poses = local_aa_fix.reshape(local_rotmat.shape[0], -1)
    # local to global
    
    # make 6D
    local_6d = utils_transform.aa2sixd(local_aa).reshape(local_rotmat.shape[0], -1)
    rotation_local_full_gt_list = local_6d #local_6d[1:]의 이유를 모르겠다.


    # SMPL과 우리 데이터셋의 Joints가 달라서 문제가 생김. Kinetic chain 활용 불가.
    #global_rotmat = local2global_pose(local_rotmat, bm.kintree_table[0].long()) ## 고쳐야 함.
    global_rotmat = world_rotmat
    
    head_rotation_global_rotmat = global_rotmat[:, [9], :, :]


    global_6d = utils_transform.matrot2sixd(global_rotmat.reshape(-1,3,3)).reshape(global_rotmat.shape[0], -1, 6)
    input_global_6d = global_6d[1:, [], :] # head : 9, right hand : 10, left hand : 14

    rotation_velocity_global_rotmat = torch.matmul(torch.inverse(global_rotmat[:-1]), global_rotmat[1:])
    rotation_velocity_global_6d = utils_transform.matrot2sixd(rotation_velocity_global_rotmat.reshape(-1, 3, 3)).reshape(rotation_velocity_global_rotmat.shape[0],-1,6)
    input_rotation_velocity_global_6d = rotation_velocity_global_6d[:, [9, 10, 14], :]

    position_global_full_gt_world = world_position
    position_head_world = world_position[:, 9, :]


    # make trans
    head_global_trans = torch.eye(4).repeat(position_head_world.shape[0], 1, 1)
    #print(head_global_trans.shape)
    head_global_trans[:, :3, :3] = head_rotation_global_rotmat.squeeze()
    head_global_trans[:, :3, 3] = position_global_full_gt_world[:, 9, :]
    
    # embed
    # ['rotation_local_full_gt_list', 'hmd_position_global_full_gt_list', 
    # 'body_parms_list', 'head_global_trans_list', 'framerate', 'gender', 
    # 'filepath', 'position_global_full_gt_world']
    head_global_trans_list = head_global_trans[1:]

    num_frames = position_global_full_gt_world.shape[0]-1

    hmd_position_global_full_gt_list = torch.cat([
        input_global_6d.reshape(num_frames, -1),
        input_rotation_velocity_global_6d.reshape(num_frames, -1),
        position_global_full_gt_world[1:, [9, 10, 14], :].reshape(num_frames, -1),
        position_global_full_gt_world[1:, [9, 10, 14], :].reshape(num_frames, -1) - position_global_full_gt_world[:-1, [9, 10, 14], :].reshape(num_frames,-1)], dim=-1)
    
    body_parms = {
        'root_orient' : local_poses[:, [27,28,29]],
        'pose_body' : torch.cat((local_poses[:, :27],local_poses[:, 29:]), dim =1)
    }

    data['rotation_local_full_gt_list'] = rotation_local_full_gt_list
    data['hmd_position_global_full_gt_list'] = hmd_position_global_full_gt_list
    data['body_parms_list'] = body_parms
    data['head_global_trans_list'] = head_global_trans_list
    data['framerate'] = 60
    data['gender'] = 'male'
    data['filepath'] = pickle_data['name'][i]
    data['position_global_full_gt_world'] = position_global_full_gt_world

    
    




01_02
05_04
07_08
07_09
07_10
08_01
102_19
102_23
103_01
103_02
103_05
105_05
105_19
105_21
105_31
105_39
105_45
105_50
105_52
106_01
106_14
106_16
106_17
107_03
108_04
108_06
108_07
108_08
108_16
108_18
108_21
108_26
10_02
10_05
111_01
111_13
111_32
111_33
113_01
113_14
118_02
118_09
118_17
120_05
120_08
120_16
122_12
122_43
122_59
122_66
122_67
123_07
125_03
125_06
126_02
126_05
127_02
127_26
131_07
132_05
132_15
132_28
132_33
132_47
133_23
133_25
135_04
136_01
136_11
136_14
136_23
136_29
137_06
137_12
137_14
137_18
139_34
13_10
13_12
13_34
141_02
141_07
141_09
141_22
142_02
142_11
142_19
143_01
143_07
143_39
143_40
144_03
144_06
144_17
15_06
15_07
15_10
16_34
17_05
17_07
18_02
18_04
18_08
18_09
18_10
18_14
20_04
22_17
22_18
22_22
27_11
29_03
29_08
29_19
30_02
30_09
30_17
31_19
35_08
35_16
35_18
35_26
35_28
35_29
36_07
36_14
36_19
36_20
36_22
40_11
41_03
41_04
54_03
54_05
54_06
54_17
54_20
55_11
55_20
55_28
60_05
60_08
62_12
62_18
62_25
63_02
63_09
63_15
63_20
63_26
64_16
64_17
64_21